# Task 1 
## Implementing Depth-First Search and Breadth-First Search 

In [1]:
from collections import deque

class Graph:
    def __init__(self):
        self.graph = {}
    
    def add_edge(self, u, v):
        if u not in self.graph:
            self.graph[u] = []
        if v not in self.graph:
            self.graph[v] = []
        self.graph[u].append(v)
        self.graph[v].append(u)
    
    def dfs_recursive(self, start):
        visited = set()
        def dfs(node):
            if node not in visited:
                visited.add(node)
                print(node, end=" ")
                for neighbor in self.graph.get(node, []):
                    dfs(neighbor)
        
        dfs(start)
        print()
    
    def dfs_iterative(self, start):
        visited = set()
        stack = [start]
        
        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                print(node, end=" ")
                for neighbor in reversed(self.graph.get(node, [])):
                    if neighbor not in visited:
                        stack.append(neighbor)
        print()
    
    def bfs(self, start):
        visited = set()
        queue = deque([start])
        
        while queue:
            node = queue.popleft()
            if node not in visited:
                visited.add(node)
                print(node, end=" ")
                for neighbor in self.graph.get(node, []):
                    if neighbor not in visited:
                        queue.append(neighbor)
        print()

def test_graph():
    g = Graph()
    
    g.add_edge(1, 2)
    g.add_edge(1, 3)
    g.add_edge(2, 4)
    g.add_edge(2, 5)
    g.add_edge(3, 6)
    g.add_edge(3, 7)
    
    print("DFS (Recursive):")
    g.dfs_recursive(1)
    
    print("DFS (Iterative):")
    g.dfs_iterative(1)
    
    print("BFS:")
    g.bfs(1)

test_graph()

DFS (Recursive):
1 2 4 5 3 6 7 
DFS (Iterative):
1 2 4 5 3 6 7 
BFS:
1 2 3 4 5 6 7 


# Task 2 
 ## Finding the Shortest Path Using Dijkstra’s Algorithm 

In [2]:
import heapq

class Graph:
    def __init__(self):
        self.graph = {}
    
    def add_edge(self, u, v, weight):
        """Add an edge to the graph with a given weight."""
        if u not in self.graph:
            self.graph[u] = []
        if v not in self.graph:
            self.graph[v] = []
        
        self.graph[u].append((v, weight))
        self.graph[v].append((u, weight))

    def dijkstra(self, start):
        """Compute shortest path from start node to all other nodes using Dijkstra's algorithm."""
        
        priority_queue = [(0, start)]
        shortest_paths = {node: float('inf') for node in self.graph}
        shortest_paths[start] = 0
        visited = set()
        
        while priority_queue:
            current_distance, current_node = heapq.heappop(priority_queue)
            
            if current_node in visited:
                continue
            
            visited.add(current_node)
            
            for neighbor, weight in self.graph.get(current_node, []):
                if neighbor in visited:
                    continue
                
                distance = current_distance + weight
                
                if distance < shortest_paths[neighbor]:
                    shortest_paths[neighbor] = distance
                    heapq.heappush(priority_queue, (distance, neighbor))
        
        return shortest_paths

    def print_shortest_paths(self, start):
        """Print the shortest path from the start node to each node."""
        shortest_paths = self.dijkstra(start)
        print(f"Shortest paths from node {start}:")
        for node, dist in shortest_paths.items():
            print(f"Node {node}: {dist}")

def test_graph():
    g = Graph()
    
    g.add_edge(1, 2, 7)
    g.add_edge(1, 3, 9)
    g.add_edge(1, 6, 14)
    g.add_edge(2, 3, 10)
    g.add_edge(2, 4, 15)
    g.add_edge(3, 4, 11)
    g.add_edge(3, 6, 2)
    g.add_edge(4, 5, 6)
    g.add_edge(5, 6, 9)
    
    g.print_shortest_paths(1)

test_graph()

Shortest paths from node 1:
Node 1: 0
Node 2: 7
Node 3: 9
Node 6: 11
Node 4: 20
Node 5: 20


# Task 3 
## Detecting Cycles in a Graph (Directed & Undirected) 

In [3]:
class Graph:
    def __init__(self, directed=False):
        self.graph = {}
        self.directed = directed

    def add_edge(self, u, v):
        if u not in self.graph:
            self.graph[u] = []
        if v not in self.graph:
            self.graph[v] = []
        self.graph[u].append(v)
        if not self.directed:
            self.graph[v].append(u)

    def find(self, parent, i):
        if parent[i] == i:
            return i
        return self.find(parent, parent[i])

    def union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)

        if xroot != yroot:
            if rank[xroot] < rank[yroot]:
                parent[xroot] = yroot
            elif rank[xroot] > rank[yroot]:
                parent[yroot] = xroot
            else:
                parent[yroot] = xroot
                rank[xroot] += 1

    def detect_cycle_undirected(self):
        parent = {}
        rank = {}

        for node in self.graph:
            parent[node] = node
            rank[node] = 0

        for node in self.graph:
            for neighbor in self.graph[node]:
                x = self.find(parent, node)
                y = self.find(parent, neighbor)

                if x == y:
                    return True
                self.union(parent, rank, x, y)
        return False

    def detect_cycle_directed(self):
        visited = set()
        rec_stack = set()

        def dfs(node):
            visited.add(node)
            rec_stack.add(node)

            for neighbor in self.graph.get(node, []):
                if neighbor not in visited and dfs(neighbor):
                    return True
                elif neighbor in rec_stack:
                    return True

            rec_stack.remove(node)
            return False

        for node in self.graph:
            if node not in visited:
                if dfs(node):
                    return True
        return False

def test_cycle_detection():
    g_undirected = Graph(directed=False)
    g_undirected.add_edge(1, 2)
    g_undirected.add_edge(2, 3)
    g_undirected.add_edge(3, 1)
    print(f"Undirected Graph has cycle: {g_undirected.detect_cycle_undirected()}")

    g_directed = Graph(directed=True)
    g_directed.add_edge(1, 2)
    g_directed.add_edge(2, 3)
    g_directed.add_edge(3, 1)
    print(f"Directed Graph has cycle: {g_directed.detect_cycle_directed()}")

    g_undirected_acyclic = Graph(directed=False)
    g_undirected_acyclic.add_edge(1, 2)
    g_undirected_acyclic.add_edge(2, 3)
    print(f"Acyclic Undirected Graph has cycle: {g_undirected_acyclic.detect_cycle_undirected()}")

    g_directed_acyclic = Graph(directed=True)
    g_directed_acyclic.add_edge(1, 2)
    g_directed_acyclic.add_edge(2, 3)
    print(f"Acyclic Directed Graph has cycle: {g_directed_acyclic.detect_cycle_directed()}")

test_cycle_detection()

Undirected Graph has cycle: True
Directed Graph has cycle: True
Acyclic Undirected Graph has cycle: True
Acyclic Directed Graph has cycle: False
